# Caching

There are multiple meanings of the word "caching" when it comes to computers and, particularly, code performance. In this section we will look at two common meanings relevant to code performance.

## Caching Variables

When certain values are calculated repeatedly, it may be worth saving their values rather than calculating them repeatedly. The more times the value is calculated and the more complex the calculation, the more viable this strategy becomes.

For example, consider the following codes which aim to calcualte:

$\sum\limits_{i=0}^{1000}\sum\limits_{j=0}^{1000}\sin{\left(\frac{i\pi}{1000}\right)}\sin{\left(\frac{j\pi}{1000}\right)} $

In [0]:
!pip install line_profiler
import math
%load_ext line_profiler

def sum_function():
  result=0

  for i in range(0, 1001):
    for j in range(0, 1001):
      result = result + math.sin(i * math.pi / 1000)*math.sin(j * math.pi / 1000)

  return(result)

%lprun -f sum_function print(sum_function())

The first thing we might notice is that we're currently performing the operation $\frac{\pi}{1000}$ 1,000,000 times and this will always have the same value. We can pre-calculate this value once and use it repeatedely:

In [0]:
!pip install line_profiler
import math
%load_ext line_profiler

def sum_function():
  result=0
  pi_over_1000 = math.pi / 1000

  for i in range(0, 1001):
    for j in range(0, 1001):
      result = result + math.sin(i * pi_over_1000)*math.sin(j * pi_over_1000)

  return(result)

%lprun -f sum_function print(sum_function())

The next thing we might notice is that there are two nested ```for``` loops. The variable ```j``` takes 1000 different values for each value ```i``` takes. Thus, we can calculate the value $\sin{\left(\frac{i\pi}{1000}\right)}$ and cache it inside the outer loop:

In [0]:
!pip install line_profiler
import math
%load_ext line_profiler

def sum_function():
  result=0
  pi_over_1000=math.pi/1000

  for i in range(0, 1001):
    sin_i = math.sin(i * pi_over_1000)
    for j in range(0, 1001):
      result = result + sin_i*math.sin(j * pi_over_1000)

  return(result)

%lprun -f sum_function print(sum_function())

This reduces the number of times we call ```math.sin``` from 2,000,000 to 1,001,000.

Finally, we might notice that we actually only call ```math.sin``` with 1,000 different values so we can actually create a list of the resultant values to cache them:

In [0]:
!pip install line_profiler
import math
%load_ext line_profiler

def sum_function():
  result=0
  pi_over_1000=math.pi/1000

  sin_values=[]

  for i in range(0, 1001):
    sin_values.append(math.sin(i * pi_over_1000))

  for sin_i in sin_values:
    for sin_j in sin_values:
      result = result + sin_i * sin_j

  return(result)

%lprun -f sum_function print(sum_function())

The resulting code calls the ```sin``` function 1,000 times and runs in about half the time compared to the original code. However, it does use more memory and is less readable.

## Caching Function Results

Often, functions will be called repeatedly with the same values passed as arguments and, thus, returning the same result. If a function is complex, it's possible to save a significant amount of time by noting a function has been called before and returning the value that was called then without performing the body of the function. For example, the following code tests a recursive function designed to calcualte the Fibonacci sequence:

In [0]:
import cProfile

def fibonacci(n):
  if n < 2:
    return(n)
  else:
    return(fibonacci(n-1) + fibonacci(n-2))

def fibonacci_range():
  for i in range(30):
    x = fibonacci(i)

cProfile.run('print(fibonacci(32))')

When we run this code we see the function is called a large number of times to calculate the desired value. We know that the function will only be called with values of ```n``` less than 32, however. This means if we could cache the results of the function with those 32 values of ```n``` we could eliminate the bodies of most of the functions and thus most of the function calls and most of the time spent.

It's possible to tell Python to cache the results of calls to a function automatically. This stores the results behind the scenes for the last few combinations of arguments used. To do this we may import the ```lru_cache``` "decorator" from the ```functools``` module and adding it to the function:

In [0]:
import cProfile
from functools import lru_cache

@lru_cache(maxsize=32)
def fibonacci(n):
  if n < 2:
    return(n)
  else:
    return(fibonacci(n-1) + fibonacci(n-2))

def fibonacci_range():
  for i in range(30):
    x = fibonacci(i)

cProfile.run('print(fibonacci(32))')

Decorators, when added to functions, modify how the function behaves. In this case, the ```lru_cache``` decorator causes the resutls of the functions for the last ```maxsize``` unique combinations of arguments provided. When one of the stored combinations of arguments is used to call the function, the cached value is returned instead of calling the function in its entirety.

In this case, the body of most function is bypassed in almost every case and, as almost all function calls are in the bodies of function, most function calls are also eliminated. This means the number of function calls is reduced from over 7,000,000 to just 33 and the run-time is also decreased from over a second to almost nothing.

This example happens to be a case where this tactic is particularly effective as we can guarantee that there will only be a small number of values passed as an argument and the number of function calls was intially very high.

## Exercise
Look at the sample code below. In the second copy of it, edit it using the principles of caching described above to make it run more quickly. Note that there are two sample solutions showing two different levels of caching.

In [0]:
# The original
import cProfile
import math

def triangle_wave_next(value, increasing, step, maximum):
  # This function gives the next point from a set of samples along a trinagle wave function (see https://en.wikipedia.org/wiki/Triangle_wave)
  # The gradient of the function is always +/-1
  # Value is current value
  # Increasing is a boolean which tells us if we're currently on an upward or downward section of the sawtooth
  # Step is how much further along the next sample point is
  # The traingle wave function osciallate between +/-maximum
  if increasing:
    new_value=value + step
    if new_value > maximum:
      new_value = 2 * maximum - new_value
      increasing = False
  else:
    new_value=value - step
    if new_value < -maximum:
      new_value = - 2 * maximum - new_value
      increasing = True

  return(new_value, increasing)

def sum_cos_traingle_wave(step, maximum):
  # This function progressiely advances through the traignle function using the variable triangle_wave_value
  # It uses this value as an argument for the cosine function and sums the resultant value
  triangle_wave_value=0
  triangle_wave_increasing=True

  sum = 0

  for i in range(1000000):
    (triangle_wave_value, triangle_wave_increasing) = triangle_wave_next(triangle_wave_value, triangle_wave_increasing, step, maximum)

    sum = sum + math.cos(math.pi * triangle_wave_value * 0.5 / maximum)

  return(sum)

cProfile.run('print(sum_cos_traingle_wave(3, 10))')

In [0]:
# For you to edit
import cProfile
import math

def triangle_wave_next(value, increasing, step, maximum):
  # This function gives the next point from a set of samples along a trinagle wave function (see https://en.wikipedia.org/wiki/Triangle_wave)
  # The gradient of the function is always +/-1
  # Value is current value
  # Increasing is a boolean which tells us if we're currently on an upward or downward section of the sawtooth
  # Step is how much further along the next sample point is
  # The traingle wave function osciallate between +/-maximum
  if increasing:
    new_value=value + step
    if new_value > maximum:
      new_value = 2 * maximum - new_value
      increasing = False
  else:
    new_value=value - step
    if new_value < -maximum:
      new_value = - 2 * maximum - new_value
      increasing = True

  return(new_value, increasing)

def sum_cos_traingle_wave(step, maximum):
  # This function progressiely advances through the traignle function using the variable triangle_wave_value
  # It uses this value as an argument for the cosine function and sums the resultant value
  triangle_wave_value=0
  triangle_wave_increasing=True

  sum = 0

  for i in range(1000000):
    (triangle_wave_value, triangle_wave_increasing) = triangle_wave_next(triangle_wave_value, triangle_wave_increasing, step, maximum)

    sum = sum + math.cos(math.pi * triangle_wave_value * 0.5 / maximum)

  return(sum)

cProfile.run('print(sum_cos_traingle_wave(3, 10))')

In [0]:
#@title
# The first sample solution
# The first optimisation is to put an lru_cache around triangle_wave_next.
# We know there are only 40 combinations of increasing and value that are possible (-10 to +20 and True/False with the exception that +10 will come alongside True and -10 will only come alongisde False)
# This means we can set the maxsize to 42
# This reduces the number of calls to this function from 1,000,000 to 40
import cProfile
import math
from functools import lru_cache

@lru_cache(maxsize=40)
def triangle_wave_next(value, increasing, step, maximum):
  if increasing:
    new_value=value + step
    if new_value > maximum:
      new_value = 2 * maximum - new_value
      increasing = False
  else:
    new_value=value - step
    if new_value < -maximum:
      new_value = - 2 * maximum - new_value
      increasing = True

  return(new_value, increasing)

def sum_cos_traingle_wave(step, maximum):
  triangle_wave_value=0
  triangle_wave_increasing=True

  sum = 0

  for i in range(1000000):
    (triangle_wave_value, triangle_wave_increasing) = triangle_wave_next(triangle_wave_value, triangle_wave_increasing, step, maximum)

    sum = sum + math.cos(math.pi * triangle_wave_value * 0.5 / maximum)

  return(sum)

cProfile.run('print(sum_cos_traingle_wave(3, 10))')

In [0]:
#@title
# The first sample solution
# The second optimisation is to notice that triangle_wave_value in sum_cos_triangle_wave will only ever take 21 values and so we can cache the values returned by the cosine function.
# We further note that, as the cosine function is symmetric, we actually only need to store the values of cosine returned for triangle_wave_value between 0 and 10.
# Although this method adds the overhead of a call to abs() it eliminates almsot all calls to math.cos and the arithmetic that was previously found in the argument of that function.
import cProfile
import math
from functools import lru_cache

@lru_cache(maxsize=40)
def triangle_wave_next(value, increasing, step, maximum):
  if increasing:
    new_value=value + step
    if new_value > maximum:
      new_value = 2 * maximum - new_value
      increasing = False
  else:
    new_value=value - step
    if new_value < -maximum:
      new_value = - 2 * maximum - new_value
      increasing = True

  return(new_value, increasing)

def sum_cos_traingle_wave(step, maximum):
  cos_values=[]
  for i in range(11):
    cos_values.append(math.cos(math.pi * i * 0.5 / maximum))

  triangle_wave_value=0
  triangle_wave_increasing=True

  sum = 0

  for i in range(1000000):
    (triangle_wave_value, triangle_wave_increasing) = triangle_wave_next(triangle_wave_value, triangle_wave_increasing, step, maximum)

    sum = sum + cos_values[abs(triangle_wave_value)]

  return(sum)

cProfile.run('print(sum_cos_traingle_wave(3, 10))')